In [ ]:
import geopandas
import json
import shapely
import shapely.geometry
from shapely.ops import transform
from shapely.geometry import Point
from shapely.geometry import Polygon
from pystac_client import Client 
from collections import defaultdict
import xarray
import rasterio as rio
import rioxarray
import os
import fiona
import numpy as np

import os
import geopandas
import pandas as pd
import fiona
import numpy as np


In [ ]:
##### START OPTIONS #####
yoi = [2021]
#toi = ['15STT']
cloud_thres = 5
valid_months = [3,4,5,6,7,8,9]
root_path = "/data/"

## file paths
spath = root_path + f"CDL_HLS_dataframe{yoi[0]}"
image_index_file = root_path + f"image_index{yoi[0]}"
chip_file =  root_path + "chip_bbox.geojson"
chip_csv = root_path + "chip_tracker.csv"
kml_file = root_path + 'sentinel_tile_grid.kml'
cdl_reclass_csv = root_path + "cdl_freq.csv"
tile_tracker_csv = root_path + "tile_tracker.csv"

## folder paths
chip_dir = root_path + 'chips/'
tif_dir = root_path + 'tif/'
chip_dir_binary = root_path + 'chips_binary/'
chip_dir_multi = root_path + 'chips_multi/'

chip_dir_filt = root_path + 'chips_filtered/'
chip_dir_binary_filt = root_path + 'chips_binary_filtered/'
chip_dir_multi_filt = root_path + 'chips_multi_filtered/'

chip_qa_dir = root_path + 'chips_qa/'

#####  END OPTIONS  #####

In [ ]:
# field = geopandas.read_file("/cdl_training_data/data/chip_bbox.geojson")
# field

In [ ]:
with open("/cdl_training_data/data/chip_bbox.geojson", "r") as file:
    chips = json.load(file)
    
chip_ids = []
chip_x = []
chip_y = []
for item in chips['features']:
    #print(item)
    chip_ids.append(item['properties']['id'])
    chip_x.append(item['properties']['center'][0])
    chip_y.append(item['properties']['center'][1])


#chip_ids = a.fea
#print(a['features'][0]['properties']['center'])

In [ ]:
# Load the HLS tiles and place there coordinates into a numpy array for processing later

fiona.drvsupport.supported_drivers['KML'] = 'rw'
tile_src = geopandas.read_file(kml_file, driver='KML')
tile_name = []
tile_x = []
tile_y = []
for tile_ind in range(tile_src.shape[0]):
    tile_name.append(tile_src.iloc[tile_ind].Name)
    tile_x.append(tile_src.iloc[tile_ind].geometry.centroid.x)
    tile_y.append(tile_src.iloc[tile_ind].geometry.centroid.y)
tile_name = np.array(tile_name)
tile_x = np.array(tile_x)
tile_y = np.array(tile_y)
tile_src = pd.concat([tile_src, tile_src.bounds], axis = 1)
#del tile_src
tile_src.head(5)

In [ ]:
def find_tile(x,y):
# Identify closest tile
    s = (tile_x - x)**2+(tile_y - y)**2
    tname = tile_name[np.argmin(s)]
    return(tname)

In [ ]:
chip_df = pd.DataFrame({"chip_id" : chip_ids, "chip_x" : chip_x, "chip_y" : chip_y})
chip_df['tile'] = chip_df.apply(lambda row : find_tile(row['chip_x'], row['chip_y']), axis = 1)
chip_df.tail(5)

In [ ]:
chips['features'][6398]

In [ ]:
tiles = chip_df.tile.unique().tolist()
tiles

In [ ]:
tiles.remove('01SBU')
tiles

In [ ]:
STAC_URL = 'https://cmr.earthdata.nasa.gov/stac'
catalog = Client.open(f'{STAC_URL}/LPCLOUD/')

In [ ]:
for current_tile in tiles[0:2]:
    print(current_tile)
    ## find single chip for current tile
    chip_df_filt = chip_df.loc[chip_df.tile == current_tile]#.reset_index()
    print(chip_df_filt)
    first_chip_id = chip_df_filt.chip_id.iloc[0]
    print(first_chip_id)
    first_chip_index_in_json = chip_ids.index(first_chip_id)
    print(first_chip_index_in_json)
    roi = chips['features'][first_chip_index_in_json]['geometry']
    print(roi)
    search = catalog.search(
        collections = ['HLSS30.v2.0'],
        intersects = roi,
        datetime = '2021-03/2021-10',
    ) 
    num_results = search.matched()
    item_collection = search.get_all_items()
    print(num_results)
    print(list(item_collection))


In [ ]:
chip_ids.index('079_078')    

In [ ]:
extract_attribute_with_value(chips, 'id', '000_001')

In [ ]:
chips['features']

In [ ]:
fieldShape = field['geometry'][0]
fieldShape

In [ ]:
type(fieldShape)

In [ ]:
roi = json.loads(geopandas.GeoSeries([fieldShape]).to_json())
roi

In [ ]:
roi['features'][0]['geometry']['coordinates'][0] = roi['features'][0]['geometry']['coordinates'][0][::-1]

In [ ]:
search = catalog.search(
    collections = ['HLSS30.v2.0'],
    intersects = roi['features'][0]['geometry'],
    datetime = '2021-03/2021-10',
) 

In [ ]:
search.matched()

In [ ]:
item_collection = search.get_all_items()

In [ ]:
item_collection

In [ ]:
len(item_collection)

In [ ]:
item_collection[0]

In [ ]:
data_url = item_collection[0].assets['B04'].href
data_url

In [ ]:
s30_bands = ['B8A', 'B04', 'B02', 'Fmask'] 

In [ ]:
band_links = []

for i in item_collection:
    if i.properties['eo:cloud_cover'] <= cloudcover:
        if i.collection_id == 'HLSS30.v2.0':
            # print(i.properties['eo:cloud_cover'])
            bands = s30_bands
        for a in i.assets:
            if any(b==a for b in bands):
                band_links.append(i.assets[a].href)

In [ ]:
band_links[:10]

In [ ]:
tile_dicts = defaultdict(list)

In [ ]:
for l in band_links:
    tile = l.split('.')[-6]
    tile_dicts[tile].append(l)

In [ ]:
tile_dicts.keys()

In [ ]:
tile_dicts['T14SMF'][:5]

In [ ]:
tile_links_T14SMF = tile_dicts['T14SMF']

In [ ]:
tile_links_T14SMF[:10]

In [ ]:
bands_dicts = defaultdict(list)

In [ ]:
for b in tile_links_T14SMF:
    band = b.split('.')[-2]
    bands_dicts[band].append(b)

In [ ]:
bands_dicts.keys()